# Map

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_4.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval,
        'GEOLOC_CIUDAD_AUTOR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 321 ms, sys: 7.96 ms, total: 329 ms
Wall time: 328 ms


In [3]:
autores_df.shape

(5364, 14)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO,GEOLOC_CIUDAD_AUTOR
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó,True,Oscar Collazos,"(6.222551999999999, -77.40115399999999)"
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogotá,Cundinamarca,True,Rafael Pombo,"(4.710988599999999, -74.072092)"
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogotá,Cundinamarca,True,Rafael Pombo,"(4.710988599999999, -74.072092)"
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogotá,Cundinamarca,True,Vladdo,"(4.710988599999999, -74.072092)"
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Aguadas,Caldas,True,Javier Ocampo López,"(5.608334, -75.453135)"


## Graph

### Rutas

In [5]:
rutas_df = autores_df[autores_df['PAIS_EJEMPLAR'] != 'Colombia']

In [6]:
len(rutas_df)

760

In [7]:
rutas_graph = nx.Graph()

for i, x in rutas_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not rutas_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        rutas_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        rutas_graph.node[ciudad]['weight'] += 1

In [8]:
rutas_graph.number_of_nodes()

64

### Ciclos

In [9]:
ciclos_df = autores_df[autores_df['PAIS_EJEMPLAR'] == 'Colombia']

In [10]:
len(ciclos_df)

4604

In [11]:
ciclos_graph = nx.Graph()

for i, x in ciclos_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not ciclos_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        ciclos_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        ciclos_graph.node[ciudad]['weight'] += 1

In [12]:
ciclos_graph.number_of_nodes()

142

# Saving results

In [13]:
nx.write_gexf(ciclos_graph, 'colombianos_ciclos_graph.gexf')
nx.write_gexf(rutas_graph, 'colombianos_rutas_graph.gexf')